# Xmen Experiment Classes

## Defining Experiments

```python
from xmen.experiment import Experiment
import os
import time
from typing import List


class BaseExperiment(Experiment):
    """A basic python experiment demonstrating the features of the xmen api."""

    # Parameters are defined as attributes in the class body with the
    # @p identifier

    t = 'cat'    # @p
    w = 3        # @p parameter w has a help message whilst t does not
    h: int = 10  # @p h declared with typing is very concise and neat

    # Parameters can also be defined in the __init__ method
    def __init__(self, *args, **kwargs):
        super(BaseExperiment, self).__init__(*args, **kwargs)

        self.a: str = 'h'  # @p A parameter
        self.b: int = 17   # @p Another parameter

        # Normal attributes are still allowed
        self.c: int = 5    # This is not a parameter


class AnotherExperiment(BaseExperiment):
    m: str = 'Another value'  # @p Multiple inheritance example
    p: str = 'A parameter only in Another Experiment '  # @p


class AnExperiment(BaseExperiment):
                    #     |
                    # Experiments can inherit from other experiments
                    # parameters are inherited too
    """An experiment testing the xmen experiment API. The __docstring__ will
    appear in both the docstring of the class __and__ as the prolog in the
    command line interface."""

    # Feel free to define more parameters
    x: List[float] = [3., 2.]  # @p Parameters can be defined cleanly as class attributes
    y: float = 5  # @p This parameter will have this
    # Parameters can be overridden
    a: float = 0.5  # a's default and type will be changed. Its help will be overridden
    b: int = 17  # @p b's help will be changed

    m: str = 'Defined in AnExperiment'  # @p m is defined in AnExperiment

    def run(self):
        # Experiment execution is defined in the run method
        print(f'The experiment state inside run is {self.status}')

        # recording messaging is super easy
        self.message({'time': time.time()})

        # Each experiment has its own unique directory. You are encourage to
        # write out data accumulated through the execution (snapshots, logs etc.)
        # to this directory.
        with open(os.path.join(self.directory, 'logs.txt'), 'w') as f:
            f.write('This was written from a running experiment')

    def debug(self):
        self.a = 'In debug mode'
        return self

    @property
    def h(self):
        return 'h has been overloaded as propery and will no longer' \
               'considered as a parameter'


# Experiments can inheret from multiple classes
class MultiParentsExperiment(AnotherExperiment, AnExperiment):
    pass
```

Note: the above is a copy of what is defined in ``xmen.examplex.inheritance``

In [1]:
from xmen.examples.inheritance import AnExperiment, AnotherExperiment, MultiParentsExperiment
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'limit_output': 10})

## Automatic Documentation

In [2]:
# documentation is automatically added to the class
help(AnExperiment)

Help on class AnExperiment in module xmen.examples.inheritance:

class AnExperiment(BaseExperiment)
 |  AnExperiment(*args, **kwargs)
 |  
 |  An experiment testing the xmen experiment API. The __docstring__ will
 |      appear in both the docstring of the class __and__ as the prolog in the
 |      command line interface.
 |  
 |  Parameters:
 |      BaseExperiment
 |       t: None (default=cat)
 |       w: parameter w has a help message whilst t does not (default=3)
 |       a (float): A parameter (default=0.5)
 |      AnExperiment
 |       b: int=17 ~ b's help will be changed
 |       x: List[float]=[3., 2.] ~ Parameters can be defined cleanly as class attributes
 |       y: float=5 ~ This parameter will have this
 |       m: str='Defined in AnExperiment' ~ m is defined in AnExperiment
 |  
 |  Method resolution order:
 |      AnExperiment
 |      BaseExperiment
 |      xmen.experiment.Experiment
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  debug(self)
 |      Inhe

## Configuring Experiments

In [3]:
# to run an experiment first we initialise it
# the experiment is initialised in 'default' status
print('\nInitialising')
print('---------------------------')
exp = AnExperiment()
print(exp)


Initialising
---------------------------
status: default
created: 11-16-20-16:50:06
messages:
parameters:
  t: cat
  w: 3
  a: h
  b: 17
  x: [3.0, 2.0]
  y: 5
  m: Defined in AnExperiment


## Inheritance

In [4]:
# Experiments can inheret from multiple classes:
print('\nMultiple Inheritance')
print('----------------------')
print(MultiParentsExperiment())
print('\n Parameters defaults, helps and values are '
      'inherited according to python method resolution order '
      '(i.e left to right). Note that m has the value '
      'defined in Another Experiment')


Multiple Inheritance
----------------------
status: default
created: 11-16-20-16:50:07
messages:
parameters:
  t: cat
  w: 3
  a: h
  b: 17
  m: Another value
  p: A parameter only in Another Experiment 
  x: [3.0, 2.0]
  y: 5

 Parameters defaults, helps and values are inherited according to python method resolution order (i.e left to right). Note that m has the value defined in Another Experiment


## Configuring

In [5]:
# whilst the status is default the parameters of the
# experiment can be changed
print('\nConfiguring')
print('------------ ')
exp = AnExperiment()
exp.a = 'hello'
exp.update({'t': 'dog', 'w': 100})
# Note parameters are copied from the class during
# instantiation. This way you don't need to worry
# about accidentally changing the mutable class
# types across the entire class.
exp.x += [4.]
print(exp)
assert AnExperiment.x == [3., 2.]
# If this is not desired (or neccessary) initialise
# use exp = AnExperiment(copy=False)


Configuring
------------ 
status: default
created: 11-16-20-16:50:07
messages:
parameters:
  t: dog
  w: 100
  a: hello
  b: 17
  x: [3.0, 2.0, 4.0]
  y: 5
  m: Defined in AnExperiment


## Registering

In [6]:
print('\nRegistering')
print('-------------')
# Before being run an experiment needs to be registered
# to a directory
exp.register('/tmp/an_experiment', 'first_experiment',
             purpose='A bit of a test of the xmen experiment api')
print(exp, end='\n')
print('\nGIT, and system information is automatically logged\n')


Registering
-------------
root: /tmp/an_experiment
name: first_experiment_4
status: registered
created: 11-16-20-16:50:07
purpose: A bit of a test of the xmen experiment api
messages:
version:
  module: xmen.examples.inheritance
  class: AnExperiment
  path: /home/robw/projects/xmen/xmen/examples
  git:
    local: /home/robw/projects/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 46b61c87b4001157a408217a1a3534f62029a1db
    branch: master
meta:
  mac: 0x708bcd585ff7
  host: yossarian
  user: robw
  home: /home/robw
parameters:
  t: dog
  w: 100
  a: hello
  b: 17
  x: [3.0, 2.0, 4.0]
  y: 5
  m: Defined in AnExperiment

GIT, and system information is automatically logged



In [7]:
# The parameters of the experiment can no longer be changed
try:
    exp.a = 'cat'
except AttributeError:
    print('Parameters can no longer be changed!', end='\n')
    pass

Parameters can no longer be changed!


## Running

In [8]:
# An experiment can be run either by...
# (1) calling it
print('\nRunning (1)')
print('-------------')
exp()


Running (1)
-------------
The experiment state inside run is running


In [9]:
# (2) using it as a context. Just define a main
#     loop like you normally would
print('\nRunning (2)')
print('-------------')
with exp as e:
    # Inside the experiment context the experiment status is 'running'
    print(f'Once again the experiment state is {e.status}')
    # Write the main loop just as you normally would"
    # using the parameters already defined
    results = dict(sum=sum(e.x), max=max(e.x), min=min(e.x))
    # Write results to the expeirment just as before
    e.message(results)


Running (2)
-------------
Once again the experiment state is running


In [10]:
# All the information about the current experiment is
# automatically saved in the experiments root directory
# for free
print(f'\nEverything ypu might need to know is logged in {exp.directory}/params.yml')
print('-----------------------------------------------------------------------------------------------')
print('Note that GIT, and system information is automatically logged\n'
      'along with the messages')
with open('/tmp/an_experiment/first_experiment/params.yml', 'r') as f:
    print(f.read())


Everything ypu might need to know is logged in /tmp/an_experiment/first_experiment_4/params.yml
-----------------------------------------------------------------------------------------------
Note that GIT, and system information is automatically logged
along with the messages
_created: 11-16-20-16:27:41  # _created: str=now_time ~ The date the experiment was created
_messages: # _messages: Dict[Any, Any]={} ~ Messages left by the experiment
  time: 1605544061.5061884
  sum: 9.0
  max: 4.0
  min: 2.0
_meta: # _meta: Optional[Dict]=None ~ The global configuration for the experiment manager
  mac: '0x708bcd585ff7'
  host: yossarian
  user: robw
  home: /home/robw
_name: first_experiment # _name: Optional[str]=None ~ The name of the experiment (under root)
_purpose: A bit of a test of the xmen experiment api # _purpose: Optional[str]=None ~ A description of the experiment purpose
_root: /tmp/an_experiment # _root: Optional[str]=None ~ The root directory of the experiment
_status: finishe